# PowerShell import from Excel / Transform / export it back

## Get some source data

This is the English Premier League results for the 2021/2 season. 
20 teams play host the other 19 at home = 380 games.  

In [ ]:
$games = Invoke-RestMethod -Headers @{
  "authority"="footballapi.pulselive.com"
  "accept-language"="en-GB,en;q=0.9,en-US;q=0.8"
  "if-none-match"="W/`"05cdc489781132ec0f1a820ebe35c7999`""
  "origin"="https://www.premierleague.com"
  "referer"="https://www.premierleague.com/"
} -Uri "https://footballapi.pulselive.com/football/fixtures?comps=1&compSeasons=418&teams=1,2,130,131,43,4,6,7,9,26,10,11,12,23,14,20,21,33,25,38&page=0&pageSize=400&sort=desc&statuses=C&altIds=true"



Sample record 

In [ ]:
$games.content[0] | Out-string -Width 220

Transform the date, and get the ground, the teams, their scores and whether it is a home win, away win or draw

In [ ]:
$games.content[0]  | select @{n="Gameweek";      e={$_.Gameweek.GameWeek}}, 
                             @{n="Kickoff";      e={[datetime]::UnixEpoch.AddMilliseconds($_.kickoff.millis).ToLocalTime()}},
                             @{n='GroundName';   e={$_.ground.name}},
                             @{n='HomeTeamName'; e={$_.Teams[0].Team.Name}},
                             @{n='HomeTeamScore';e={$_.Teams[0].Score}},
                             @{n='AwayTeamName'; e={$_.Teams[1].Team.Name}},
                             @{n='AwayTeamScore';e={$_.Teams[1].Score}}, 
                             outcome

And send it to Excel 

In [ ]:
Remove-Item .\EPL21-22-raw.xlsx -ErrorAction SilentlyContinue
$games.content | Select-Object @{n="Gameweek";      e={$_.Gameweek.GameWeek}}, 
                               @{n="Kickoff";       e={[datetime]::UnixEpoch.AddMilliseconds($_.kickoff.millis).ToLocalTime()}},
                               @{n='GroundName';    e={$_.ground.name}},
                               @{n='HomeTeamName';  e={$_.Teams[0].Team.Name}},
                               @{n='HomeTeamScore'; e={$_.Teams[0].Score}},
                               @{n='AwayTeamName';  e={$_.Teams[1].Team.Name}},
                               @{n='AwayTeamScore'; e={$_.Teams[1].Score}}, 
                               outcome |  Sort kickoff| Export-Excel .\EPL21-22-raw.xlsx -Now 

## Read and transform the data

Turn each game into the result from each team's perspective -e.g.    
`Home win  ,   5 home goals scored 0 conceded, 3 home points `    
and   
`Away defeat , 0 away goals scored 5 conceded, 0 away points `   
380 games become 760 of these 

In [ ]:
$games   = Import-Excel .\EPL21-22-raw.xlsx 
$results = foreach ($g in $games) { 
    switch ($g.outcome) {
        "A"  {$AwayPoints = 3; $awayResult = "Win";    $HomePoints="0"; $HomeResult = "Defeat"}
        "D"  {$AwayPoints = 1; $awayResult = "Draw";   $HomePoints="1"; $HomeResult = "Draw"}
        "H"  {$AwayPoints = 0; $awayResult = "Defeat"; $HomePoints="3"; $HomeResult = "Win"}
    }
    [pscustomobject]@{
        Team             = $G.HomeTeamName
        Opponent         = $G.AwayTeamName
        HomeOrAway       = "Home"
        Date             = $G.KickOff
        GoalsFor         = $G.HomeTeamScore
        GoalsAgainst     = $G.AwayTeamScore
        GoalDifference   = $G.HomeTeamScore - $G.awayTeamScore
        AwayGoalsFor     = $null
        AwayGoalsAgainst = $null
        HomeGoalsFor     = $G.HomeTeamScore
        HomeGoalsAgainst = $G.AwayTeamScore
        Result           = $HomeResult
        AwayResult       = $null
        Homeresult       = $HomeResult
        Points           = $HomePoints
        AwayPoints       = $null
        HomePoints       = $HomePoints
        Won              = (($g.outcome -eq "H") ? 1 : $null )
        Lost             = (($g.outcome -eq "A") ? 1 : $null )
        Drawn            = (($g.outcome -eq "D") ? 1 : $null )
        HomeWon          = (($g.outcome -eq "H") ? 1 : $null )
        HomeLost         = (($g.outcome -eq "A") ? 1 : $null )
        HomeDrawn        = (($g.outcome -eq "D") ? 1 : $null )
        AwayWon          = $null
        AwayLost         = $null
        AwayDrawn        = $null
    }
        [pscustomobject]@{
        Team             = $G.AwayTeamName
        Opponent         = $G.HomeTeamName
        HomeOrAway       = "Away"
        Date             = $G.KickOff
        GoalsFor         = $G.AwayTeamScore
        GoalsAgainst     = $G.HomeTeamScore
        GoalDifference   = $G.AwayTeamScore - $G.HomeTeamScore
        AwayGoalsFor     = $G.AwayTeamScore
        AwayGoalsAgainst = $G.HomeTeamScore
        HomeGoalsFor     = $null
        HomeGoalsAgainst = $null
        Result           = $AwayResult
        AwayResult       = $AwayResult
        Homeresult       = $null
        Points           = $AwayPoints
        AwayPoints       = $AwayPoints
        HomePoints       = $Null
        Won              = (($g.outcome -eq "A") ? 1 : $null )
        Lost             = (($g.outcome -eq "H") ? 1 : $null )
        Drawn            = (($g.outcome -eq "D") ? 1 : $null )
        HomeWon          = $null
        HomeLost         = $null
        HomeDrawn        = $null
        AwayWon          = (($g.outcome -eq "A") ? 1 : $null )
        AwayLost         = (($g.outcome -eq "H") ? 1 : $null )
        AwayDrawn        = (($g.outcome -eq "D") ? 1 : $null )
    }
}

In [ ]:
$results[0,1]  |Out-string

In [ ]:
Get-item .\EPL21-22-processed.xlsx,  .\EPL21-22-Table.xlsx   -ErrorAction SilentlyContinue | Remove-Item
$results | Export-Excel -Path .\EPL21-22-processed.xlsx -FreezeTopRow -TableName TeamsGames -WorksheetName Results -ClearSheet -TableStyle Medium6 -AutoSize -Show -PivotTableName WinLoss -PivotRows Team -PivotColumn Result -PivotData @{"Date"="Count"} -PivotChartType ColumnStacked

### With the pivot module we can transform this into the league table

In [ ]:
ipmo ~\Documents\GitHub\PowerShellPivot\PowerShellPivot.psd1 -force

In [ ]:
$table = $results | Get-Subtotal -ValueName HomeWon,          AwayWon,          Won,
                                            HomeLost,         AwayLost,         Lost,
                                            HomeDrawn,        AwayDrawn,        Drawn,
                                            HomeGoalsAgainst, AwayGoalsAgainst, GoalsAgainst,
                                            HomeGoalsFor,     AwayGoalsFor,     GoalsFor,
                                            GoalDifference,
                                            HomePoints,       AwayPoints,       Points   -NoSuffix -GroupByName Team -Sum | 
                        Sort-Object Points, GoalDifference,   GoalsFor -Descending

In [ ]:
$table | Select-Object -Property * -ExcludeProperty Home*,Away* | ft -a *  | out-string -width 240 | %{$_ -replace '(\d+)\.00','   $1'} 

And save that to excel with some extra columns  (e.g Total For/Total Against = "Goal average" )

In [ ]:

$table | Select-Object -Property Team, *won,*Lost,*drawn,*Goals*,
                                @{n="HomeGoalAverage"   ;e={"=HomeGoalsFor/HomeGoalsAgainst"}},
                                @{n="AwayGoalAverage"   ;e={"=AwayGoalsFor/AwayGoalsAgainst"}},
                                @{n="GoalAverage"       ;e={"=GoalsFor/GoalsAgainst"}},
                                @{n="HomeGoalDifference";e={"=HomeGoalsFor-HomeGoalsAgainst"}},
                                @{n="AwayGoalDifference";e={"=AwayGoalsFor-AwayGoalsAgainst"}},
                                GoalDifference, *Points  |
            Export-Excel -Path .\EPL21-22-Table.xlsx -AutoNameRange -WorksheetName Table1 -TableStyle Medium6  -ClearSheet  -AutoSize -Activate -show





### Or we can output a sheet designed to total with a pivot table

In [ ]:
 
$numbers  = $results |  Select-Object Team,Result,HomeOrAway, @{n="Value";e="Points"},         @{n="Type";e={"Points"}}
$numbers += $results |  Select-Object Team,Result,HomeOrAway, @{n="Value";e="GoalDifference"}, @{n="Type";e={"GoalDifference"}} 
$numbers += $results |  Select-Object Team,Result,HomeOrAway, @{n="Value";e="GoalsFor"},       @{n="Type";e={"GoalsFor"}}
$numbers += $results |  Select-Object Team,Result,HomeOrAway, @{n="Value";e="GoalsAgainst"},   @{n="Type";e={"GoalsAgainst"}} 
$numbers += $results | Where-Object Result -eq "Defeat" | 
                        Select-Object Team,Result,HomeOrAway, @{n="Value";e={1}},               @{n="Type";e={"Lost"}} 
$numbers += $results | Where-Object Result -eq "Win" | 
                        Select-Object Team,Result,HomeOrAway, @{n="Value";e={1}},               @{n="Type";e={"Won"}};
$numbers += $results | Where-Object Result -eq "Draw" | 
                        Select-Object Team,Result,HomeOrAway, @{n="Value";e={1}},               @{n="Type";e={"Drawn"}};

$excel = $numbers | Export-Excel -PassThru -Path .\EPL21-22-Table.xlsx -FreezeTopRow -TableName BreakDown -WorksheetName LeagueNumbers -ClearSheet -TableStyle Medium6 -AutoSize 

Add-PivotTable -ExcelPackage $Excel -PivotTableName League -PivotRows Team  -PivotColumns Type  -PivotData @{"Value"="Sum"} -PivotTotals None  -Activate -SourceWorksheet $Excel.LeagueNumbers -SourceRange $Excel.LeagueNumbers.Dimension.Address

Close-ExcelPackage $excel -Show